In [9]:
from test import *

In [10]:
# Parámetros para función plot
estilos = mpf.available_styles()
tipos_graficos_disponibles = ['candle', 'ohlc', 'line']

In [11]:
###############################
# Lista con todos los tickers #
###############################

tickers = []

with open('tickers.csv', 'r') as file:
    csvfile = csv.reader(file, delimiter=',')
    for l in csvfile:
        primer = l[0]
        tickers.append(primer[2:-1])

del tickers[0]

In [13]:
def ejecutar_accion(Funciones):
    if Funciones == 'Plot':
        @widgets.interact
        def manejar_empresa(txt='AAPL'):
            empresas = loc_empresa(txt)
            widgets.interact(
                plot2,
                ticker = empresas);

    elif Funciones == 'Pct_Change':
        @widgets.interact
        def manejar_empresa(txt='AAPL'):
            empresas = loc_empresa(txt)
            widgets.interact(
                pct_change1,
                ticker = empresas);

    elif Funciones == 'Cagr':
        @widgets.interact
        def manejar_empresa(txt='AAPL'):
            empresas = loc_empresa(txt)
            widgets.interact(
                cagr1,
                ticker = empresas);
            

In [14]:
widgets.interact(
    ejecutar_accion, 
    Funciones = ['-','Plot', 'Pct_Change', 'Cagr']
)

interactive(children=(Dropdown(description='Funciones', options=('-', 'Plot', 'Pct_Change', 'Cagr'), value='-'…

<function __main__.ejecutar_accion(Funciones)>